In [1]:
require('rnn')
require('hdf5')
require('optim')


f = hdf5.open("data.hdf5", "r")



train_rnn_X = f:read('train_rnn_X'):all()
train_rnn_Y = f:read('train_rnn_Y'):all()
train_X_sequence =  f:read('train_X_sequence'):all()
train_Y  = f:read('train_Y'):all()
valid_reduced_X = f:read('valid_reduced_X'):all()
valid_reduced_Y = f:read('valid_reduced_Y'):all()
windows_train = f:read('windows_train'):all()
windows_valid = f:read('windows_valid'):all()
test = f:read('test'):all()
valid_kaggle_with_spaces = f:read('valid_kaggle_with_spaces'):all()
valid_kaggle_without_spaces = f:read('valid_kaggle_without_spaces'):all()
valid_answers = f:read('valid_answers'):all()
nfeatures = (f:read('nfeatures'):all())[1]
nclasses = (f:read('nclasses'):all())[1]




f:close()

window_size = windows_train:size(2)
n_actual = nclasses

In [2]:

nclasses


2	


In [370]:
vocab_size = nfeatures
embed_dim = 15
lin_dim = 100
LT = nn.LookupTable(vocab_size, embed_dim)

lstm = nn.Sequential()
lstm:add(LT)
--lstm:add(nn.Dropout())
lstm:add(nn.SplitTable(1, 3))
seqLSTM = nn.Sequencer(nn.LSTM(embed_dim, lin_dim))
seqLSTM:add(nn.Dropout())
seqLSTM:remember('both')
lstm:add(seqLSTM)

lstm:add(nn.Sequencer(nn.Linear(lin_dim, n_actual)))

--lstm:add(nn.Sequencer(nn.LogSoftMax()))

--crit = nn.SequencerCriterion(nn.BCECriterion())

crit = nn.SequencerCriterion(nn.CrossEntropyCriterion())
params, grad_params = lstm:getParameters()
params:rand(params:size()):csub(.5):mul(.05)
count = 0

In [ ]:
seqLSTM:forget()
count = 0
function feval(w_new)
 
    --count = count + 1
    count = math.fmod(count + 1,train_rnn_X:size(1))
    if count == 0 then
        count = 1
        seqLSTM:forget()
    end
    
    x_d = (train_rnn_X[count]):t()
    y_d = (train_rnn_Y[count]):t()
    
    
    grad_params:zero()
    preds = lstm:forward(x_d)
    --print(preds[1])
    loss = crit:forward(preds, y_d)
    grad_out = crit:backward(preds, y_d)
    
    lstm:backward(x_d, grad_out)
    
    if torch.norm(grad_params) > 5 then
        nn.Normalize(2):forward(grad_params)
        grad_params:mul(5)
    end
    
    return loss, grad_params
end

for i = 1,40000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,params)
    if i % 10 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] , count)
        
        -- print(sgd_params)
    end
end

loss for iteration 10 is 0.98731130227461	10	


loss for iteration 20 is 0.65161224485017	20	


loss for iteration 30 is 1.0223166355632	30	


loss for iteration 40 is 0.98401989035034	40	


loss for iteration 50 is 1.0449116013892	50	


...

loss for iteration 330 is 1.0212359581147	330	


loss for iteration 340 is 0.67817013814407	340	


loss for iteration 350 is 0.71473707370769	350	


loss for iteration 360 is 0.81539030156475	360	


loss for iteration 370 is 0.93442374663543	370	


In [360]:
function perplexity(input_set, output_set)
    sum = 0
    seqLSTM:forget()
    distributions = torch.Tensor(input_set:size(1),2)
    for i = 1,input_set:size(1) do
        
        a = lstm:forward(input_set:sub(i,i))
        
        distribution = (nn.LogSoftMax():forward(a[1]))
        
        distributions[i] = torch.exp(distribution)
        answer = distribution[output_set[i]]
        sum = sum + answer
        
    end
    return torch.exp(-sum/input_set:size(1)), distributions
end

In [414]:
a,b = perplexity((valid_reduced_X:sub(1,100000)), valid_reduced_Y)
print(a)


1.168772848808	


In [415]:
a,b = perplexity((train_X_sequence:sub(1,10000)), train_Y:sub(1,100000))
print(a)


1.092589718542	


In [304]:
function predict(input_set, output_set, cutoff)
    kaggle = torch.zeros(input_set:size(1))
    for i = 1,input_set:size(1) do
        count = 0
        lstm:forget()
        for j = 1, input_set:size(2) do
            if input_set[i][j] == 30 then
                
                break
            end
            a = lstm:forward(torch.Tensor{input_set[i][j]})
            
            distribution = (nn.LogSoftMax():forward(a[1]))
            distribution = torch.exp(distribution)
            
            if distribution[2] > cutoff then
                space = lstm:forward(torch.Tensor{28})
                count = count + 1
            end
            
        
        end
        kaggle[i] = count
    
    end
    if output_set ~= 0 then
        out =output_set:double()
        MSE = nn.MSECriterion():forward(kaggle, out)
    else
        out = 0
        MSE = 0
    end
    
    return kaggle, out, MSE
end

In [450]:
k, o, mse = predict(valid_kaggle_without_spaces:sub(1,1000), valid_answers:sub(1,1000),.22)



In [451]:
mse

6.525	


In [214]:
k,_,_ = predict(test, 0, .3)

In [218]:
function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Count\n")
    for i=1,scores:size(1) do
        s = tostring(i) .. "," .. tostring(scores[i])
        f:write(s .. "\n")
    end
    f:close()
end

write2file(k, 'preds.txt')

In [412]:
valid_kaggle_with_spaces:size()

 3370
  416
[torch.LongStorage of size 2]



{
  1 : DoubleTensor - size: 1x2
  2 : DoubleTensor - size: 1x2
  3 : DoubleTensor - size: 1x2
  4 : DoubleTensor - size: 1x2
  5 : DoubleTensor - size: 1x2
  6 : DoubleTensor - size: 1x2
  7 : DoubleTensor - size: 1x2
  8 : DoubleTensor - size: 1x2
  9 : DoubleTensor - size: 1x2
  10 : DoubleTensor - size: 1x2
}


bad argument #2 to '?' (out of range at /Users/austinshin/torch/pkg/torch/generic/Tensor.c:890)
stack traceback:
	[C]: at 0x0cf96ad0
	[C]: in function '__index'
	[string "function predict(input_set, output_set, cutof..."]:11: in function 'f'
	[string "local f = function() return predict(valid_kag..."]:1: in main chunk
	[C]: in function 'xpcall'
	...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:209: in function <...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:173>
	...s/austinshin/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...ustinshin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...s/austinshin/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010cc0dbc0: 